Importo los archivos los archivos y tomo las caracteristicas de interes



In [ ]:
!pip install arch

In [ ]:
import matplotlib.pyplot as plt
import h5py
import numpy as np
from arch.unitroot import ADF, PhillipsPerron
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.signal import periodogram
from scipy.interpolate import interp1d
from scipy.signal import butter, lfilter

In [ ]:
# Open hdf5 files
filename_h1 = 'h1_4khz_32s.hdf5'
filename_l1 = 'l1_4khz_32s.hdf5'

with h5py.File(filename_h1, 'r') as f_h1:
    h1_s = f_h1['strain/Strain'][...]
    h1_fs = 1 / f_h1['strain/Strain'].attrs['Xspacing']
    h1_st = f_h1['meta/GPSstart'][()]

with h5py.File(filename_l1, 'r') as f_l1:
    l1_s = f_l1['strain/Strain'][...]
    l1_fs = 1 / f_l1['strain/Strain'].attrs['Xspacing']
    l1_st = f_l1['meta/GPSstart'][()]

In [ ]:
h1_st

In [ ]:
len(l1_s)

Realizo un EDA

In [ ]:
gw_time = 1126259462.442
gw_st = gw_time - h1_st
gw_large = 0.2
time = np.arange(0,h1_s.shape[0]/h1_fs, 1/h1_fs)

In [ ]:
gw_st

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(time, h1_s, linewidth=0.5, color= "black")
ax1.set_title('h1')
ax1.set_xlabel('Time (s)')

ax2.plot(time, l1_s, linewidth=0.5, color= "orange")
ax2.set_title('l1')
ax2.set_xlabel('Time (s)')

plt.tight_layout()
plt.show()

Separo una porcion del espectro con el evento y analizo despues el ruido por separado

In [ ]:
N = 1
h1_gw = h1_s[(time >= gw_st-N) & (time <= gw_st + N)]
l1_gw = l1_s[(time >= gw_st-N) & (time <= gw_st + N)]
time_gw = time[(time >= gw_st-N) & (time <= gw_st + N)]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(time_gw, h1_gw, linewidth=0.5, color= "yellow")
ax1.set_title('h1_gw')
ax1.set_xlabel('Time (s)')

ax2.plot(time_gw, l1_gw, linewidth=0.5)
ax2.set_title('l1_gw')
ax2.set_xlabel('Time (s)')

plt.tight_layout()
plt.show()

Ahora separo el ruido para estudiar estacionareidad

In [ ]:
h1_ruido = h1_s[(time > gw_st + N)]
l1_ruido = l1_s[(time > gw_st + N)]
time_ruido = time[(time > gw_st + N)]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(time_ruido, h1_ruido, linewidth=0.5, color= "red")
ax1.set_title('h1_ruido')
ax1.set_xlabel('Time (s)')

ax2.plot(time_ruido, l1_ruido, linewidth=0.5)
ax2.set_title('l1_ruido')
ax2.set_xlabel('Time (s)')

plt.tight_layout()
plt.show()

TEST DE ESTACIONAREIDAD

Se multiplica por una constante grande, ya que debido a los valores de la señal hace underflow en algunos casos dando resultados erroneos

In [ ]:
def EsEstacionario(signal):
  #PhillipsPerron Test
  phpr_result = PhillipsPerron(signal)
  phpr_p_value = phpr_result.pvalue
  phpr_test_result = "Stationary" if phpr_p_value < alpha else "Not stationary"
  print("Augmented Dickey-Fuller Test:", phpr_test_result)
  # Augmented Dickey-Fuller Test
  adf_result = ADF(signal)
  adf_p_value = adf_result.pvalue #adf_result[1]
  adf_test_result = "Stationary" if adf_p_value < alpha else "Not stationary"
  print("Augmented Dickey-Fuller Test:", adf_test_result)
  # Kwiatkowski-Phillips-Schmidt-Shin Test
  kpss_result = kpss(signal)
  kpss_p_value = kpss_result[1]
  kpss_test_result = "Stationary" if kpss_p_value > alpha else "Not stationary"
  print("Kwiatkowski-Phillips-Schmidt-Shin Test:", kpss_test_result)

In [ ]:
#Valor significativo
alpha = 0.05

In [ ]:
len(l1_ruido)

In [ ]:
len(h1_ruido)

In [ ]:
EsEstacionario(l1_s*1e30)

In [ ]:
EsEstacionario(h1_s*1e30)

In [ ]:
EsEstacionario(l1_ruido*1e30)

In [ ]:
EsEstacionario(h1_ruido*1e30)

Da estacionario, restamos entonces las medias a cada señal.

In [ ]:
h1_ruido = h1_ruido - np.mean(h1_ruido)
l1_ruido = l1_ruido - np.mean(l1_ruido)

Periodograma

In [ ]:
def periodogram(x, win):
    N = len(x)
    w = scipy.signal.get_window(Nx = N, window = win)
    Rx = (scipy.fft.fft(x*w))**2 * (1/N)
    return Rx

In [ ]:
Rx = periodogram(h1_ruido, 'bartlett')
Rx2 = periodogram(h1_ruido, 'parzen')
x_ax = np.linspace(0,0.5, len(Rx)) * h1_fs
plt.plot(x_ax, 10*np.log10(Rx), 'blue')
plt.plot(x_ax, 10*np.log10(Rx2), 'red')
plt.title('Periodogram - N= ' + str(len(h1_ruido)))
plt.grid()
plt.xscale("log")
plt.show()

Welch Bartlett

In [ ]:
def autocorr(x, L):
    '''N = len(x)
    rx = np.zeros(N)
    for l in range(L):
        sum = 0
        for i in range(N-l):
            sum += (x[i]*x[i+l])/N
        rx[l]=sum
    rx2 = (rx[1:])[::-1]
    return np.concatenate([rx2, rx])'''
    return scipy.signal.correlate(x,x, 'full')/len(x)


def apply_window(rx, L, win):
    N = len(rx)
    window = scipy.signal.get_window(Nx = 2*L+1, window = win)
    pw = int((N-len(window))/2)

    padded_window = np.pad(array = window, pad_width=pw)
    return rx*padded_window


def blackman_tuckey(signal, L, w):
    rx = autocorr(signal, L)
    N = len(h1_ruido)
    Rx_smoothing = abs(scipy.fft.fft(apply_window(rx, L, w)))
    return Rx_smoothing[:len(h1_ruido)]

In [ ]:
def welch_bartlett(signal, w, segment_size, overlap = 0):

    X_prom = np.zeros(segment_size)
    N = len(signal)
    i=0
    segment_count = 0
    d = int(segment_size*(1-overlap))
    while i+segment_size<N:
        segment = signal[i:i+segment_size]
        s_len = len(segment)
        Xr = abs(scipy.fft.fft(segment*scipy.signal.get_window(Nx = s_len, window = w)))
        X_prom += np.power(Xr, 2)
        i=i+d
        segment_count+=1


    X_prom /= (segment_count*segment_size)

    return X_prom[:int(segment_size/2)]

In [ ]:
w = 'parzen'
L = 15000
Rx_pa = welch_bartlett(h1_ruido, w, L, overlap=0.25)
#plt.plot(np.array(range(len(Rx_pa)))/len(Rx_pa),20*np.log10(Rx_pa), label = L)
plt.title('Welch-Bartlett - win: ' + w)
#plt.grid()
#plt.xscale("log")

w = 'parzen'
L = 5000
Rx_pa = welch_bartlett(h1_ruido, w, L, overlap=0.25)
x_ax = np.linspace(0,0.5, len(Rx_pa)) * h1_fs
plt.plot(x_ax, 10*np.log10(Rx_pa), label = L)
#plt.title('Welch-Bartlett - win: ' + w)
#plt.grid()
#plt.xscale("log")

w = 'parzen'
L = 10000
Rx_pa = welch_bartlett(h1_ruido, w, L, overlap=0.25)
x_ax = np.linspace(0,0.5, len(Rx_pa)) * h1_fs

plt.plot(x_ax, 10*np.log10(Rx_pa), label = L)
#plt.title('Welch-Bartlett - win: ' + w)
plt.grid()
plt.xscale("log")

plt.legend()
plt.show()

In [ ]:
L = 15000
w = 'parzen'

smoothRx = blackman_tuckey(h1_ruido, L, w)
x_ax = np.linspace(0,0.5, len(smoothRx)) * h1_fs
plt.plot(x_ax, 10*np.log10(smoothRx), label = str(L))

L =10000
w = 'parzen'
smoothRx = blackman_tuckey(h1_ruido, L, w)
plt.plot(x_ax, 10*np.log10(smoothRx), label = str(L))
plt.title('Blackman Tuckey -  L = ' + str(L) + ' - N = ' + str(len(h1_ruido)) + ' - win: ' + w)
plt.grid()
plt.xscale("log")
plt.legend()

plt.show()

In [ ]:
plt.plot(10*np.log10(smoothRx))
plt.title('Blackman Tuckey -  L = ' + str(L) + ' - N = ' + str(len(h1_ruido)) + ' - win: ' + w)
plt.grid()
plt.show()

In [ ]:
w = 'parzen'
L = 15000
Rx_pa = welch_bartlett(h1_ruido, w, L, overlap=0.25)
plt.plot(np.array(range(len(Rx_pa)))/len(Rx_pa),20*np.log10(Rx_pa), label = L)
plt.title('Welch-Bartlett - win: ' + w)
#plt.grid()
#plt.xscale("log")

w = 'parzen'
L = 5000
Rx_pa = welch_bartlett(h1_ruido, w, L, overlap=0.25)
plt.plot(np.array(range(len(Rx_pa)))/len(Rx_pa),20*np.log10(Rx_pa), label = L)
#plt.title('Welch-Bartlett - win: ' + w)
#plt.grid()
#plt.xscale("log")

w = 'parzen'
L = 2000
Rx_pa = welch_bartlett(h1_ruido, w, L, overlap=0.25)
plt.plot(np.array(range(len(Rx_pa)))/len(Rx_pa), 20*np.log10(Rx_pa), label = L)
#plt.title('Welch-Bartlett - win: ' + w)
plt.grid()
plt.xscale("log")
plt.legend()
plt.show()

whitenning

hay que tomar el mejor metodo, y definir h1p_f a la frecuencia y h1p_p al espectro

In [ ]:
h1_psd = interp1d(h1p_f*h1_fs, h1p_p, kind='nearest', bounds_error=False, assume_sorted=True)
l1_psd = interp1d(l1p_f*l1_fs, l1p_p, kind='nearest', bounds_error=False, assume_sorted=True)

In [ ]:

h1_psd = np.abs(h1_psd(np.fft.rfftfreq(len(h1_s), 1/h1_fs)))**2
l1_psd = np.abs(l1_psd(np.fft.rfftfreq(len(l1_s), 1/l1_fs)))**2

# Signal whitening
h1_w = np.fft.irfft(np.fft.rfft(h1_s) / np.sqrt(h1_psd))
l1_w = np.fft.irfft(np.fft.rfft(l1_s) / np.sqrt(l1_psd))

In [ ]:
nyqh = 0.5 * h1_fs
low_h = 30
high_h = 300
lowh = low_h / nyqh
highh = high_h / nyqh
bh, ah = butter(6, [lowh, highh], btype='band')

nyql = 0.5 * l1_fs
low_l = 20
high_l = 300
lowl = low_l / nyql
highl = high_l / nyql
bl, al = butter(6, [lowl, highl], btype='band')

# Apply passband filter
h1_f_w = lfilter(bh, ah, h1_w)
l1_f_w = lfilter(bl, al, l1_w)

corr cruzada
habia que invertir uno noma

In [ ]:
interval = (time > gw_st - 0.1) & (time < gw_st + 0.1)
N = len(h1_f_w_signal[interval])
x_corr = np.correlate(h1_f_w[interval], -l1_f_w[interval], 'full')

In [ ]:
 (np.argmax(x_corr)-N)/h1_fs * 1e3